<a href="https://colab.research.google.com/github/Ang3lino/recomenderSys/blob/master/collaborativeFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np
import pandas as pd

import os
import pickle

from sortedcontainers import SortedList
from collections import Counter, defaultdict
from tqdm import tqdm  # modulo cuya finalidad es dar un feedback del progreso de algun procedimiento

In [2]:

!pip install tqdm

# User-Based Neighborhood Models
En este enfoque, user-based neighborhoods son definidos con el fin de identificar usuario similares a el usuario objetivo con el fin de predecir valoraciones que el daria a un articulo.

Para la matrix de valoraciones $m \times n$, $R = [r_{ui}]$ con $m$ usuarios y $n$ articulos. Definamos $I_u$ el conjunto de indices de articulos que el usuario $u$ ha valorado (una fila). El conjunto $I_u \cap I_v$ define las observaciones mutuamente observadas.

In [3]:
from google.colab import drive  
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def load_object(fname: str, user_count: int, item_count: int) -> defaultdict:
    fdir = 'drive/My Drive/petroleo/movielens-20m-dataset'
    fname = f'{fname}_{user_count}_{item_count}.json'
    fpath = os.path.join(fdir, 'shrinked', fname)
    with open(fpath, 'rb') as fp:
        object_ = pickle.load(fp)
    return object_

def defaultdict_set(defdict):
    return {k: set(v) for k, v in defdict.items()}


user_count = 4096
item_count = 512
user2item = load_object('user2item', user_count, item_count)
item2user = load_object('item2user', user_count, item_count)
user_item2rating = load_object('user_item2rating', user_count, item_count)

user2item = defaultdict_set(user2item)

# Pearson
Una medida que captura la similitud $Sim(u, v)$ entre los vectores de valoracion entre usuarios $u,v$ es el coeficiente de correlacion de Pearson. El primer paso es calcular las medias de valoracion $\mu_u$ para cada usuario $u$:

$$ \mu_u = \frac{\sum_{k\in I_u}r_{uk}}{|I_u|} $$

Asi, la correlacion de Parson se define como 

$$ Sim(u, v) = Pearson(u, v) = \frac{\sum_{k \in I_u \cap I_v}(r_{uk} - \mu_u)(r_{vk} - \mu_v)}
{\sqrt{\sum_{k \in I_u \cap I_v}(r_{uk} - \mu_u)^2 }  \sqrt{ \sum_{k \in I_u \cap I_v}(r_{vk} - \mu_v)^2 } 
} $$



In [0]:
def user_means(user_item2rating: dict, user2item: defaultdict) -> dict:
    means = dict() 
    for u, items in user2item.items():
        means[u] = np.mean([user_item2rating[(u, i)] for i in items])
    return means

def sim_cos(u: np.array, v: np.array):
    dot = lambda u, v: u.dot(v)
    sigma = lambda u, v: np.sqrt(dot(u, v))
    num = dot(u, v)
    den = sigma(u, u) * sigma(v, v)
    return num / den
    

def sim_pearson(u: int, v: int, means: dict, user2item: defaultdict, user_item2rating: dict):
    intersection = user2item[u] & user2item[v]
    a = np.array([user_item2rating[u, k] for k in intersection]) - means[u]
    b = np.array([user_item2rating[v, k] for k in intersection]) - means[v]
    return sim_cos(a, b)



In [6]:
means = user_means(user_item2rating, user2item)
ans = sim_pearson(1, 5, means, user2item, user_item2rating)
print(ans)

0.11996713555118083


# Prediccion
Sea $P_u(j)$ el conjunto de los k usuarios mas cercanos del usuario $u$, quienes han valorado el articulo $j$. Usuarios con un muy bajo o negativas correlaciones con el usuario $u$ son filtrados por $P_u(j)$ como una mejora heuristica. Asi, la funcion de prediccion se describe como: 

$$\hat{r_{uj}} = \mu_u + \frac{\sum_{v\in P_u(j)}Sim(u,v)(r_{vj}-\mu_v)}{\sum_{v\in P_u(j)}|Sim(u, v)|} $$

In [0]:
def euclidean_distance(u, v):
    return np.sqrt(np.sum((u - v) ** 2))

def neighbors(x_vec, y_cls, p_i, k=3):
    dist_class = [(euclidean_distance(x_i, p_i), y_cls[i]) for i, x_i in enumerate(x_vec)]
    sorted_distance = sorted(dist_class, key=lambda x: x[0])[:k]
    return [y_i for (_, y_i) in sorted_distance]

def closest_users(u: int, j: int, item2user, user_item2rating):
    users = item2user[j]
    ratings = [user_item2rating[user, j] for user in users]
    closests = neighbors(ratings, users, user_item2rating[(u, j)])
    return closests


In [8]:
u = 1
j = 2
closest_users(u, j, item2user, user_item2rating)

[405, 691, 1005]

In [0]:
def predict_rating(u: int, j: int, means: dict, user2item, item2user, user_item2rating):
    k_closest = closest_users(u, j, item2user, user_item2rating)
    similars = np.array([sim_pearson(u, v, means, user2item, user_item2rating) for v in k_closest])
    s_vj = np.array([user_item2rating[(v, j)] - means[v] for v in k_closest])
    num = np.sum(similars * s_vj ) 
    den = np.sum(list(map(abs, similars)))
    return means[u] + num/den

In [10]:
r_pred = predict_rating(u, j, means, user2item, item2user, user_item2rating)
print(r_pred)
print(user_item2rating[(u, j)])

4.629527360969084
4.0


In [0]:
def predict(user2item, item2user, user_item2rating, batch_size: int) -> dict:
    predictions = dict()
    # n = len(user2item)
    count = 0
    for u, items in tqdm(user2item.items(), total=batch_size):
        for i in items:
            predictions[(u, i)] = predict_rating(u, i, means, user2item, item2user, user_item2rating)
        count += 1
        if count > batch_size: break
    return predictions

def mse(y_true, y_pred):
    return np.sqrt(np.sum((y_true - y_pred) ** 2))

In [12]:
n_preds = 100
%time preds = predict(user2item, item2user, user_item2rating, n_preds)

100%|██████████| 100/100 [15:52<00:00,  9.94s/it]

CPU times: user 16min 3s, sys: 304 ms, total: 16min 3s
Wall time: 16min 5s


In [15]:
debug = True
acc = 0
count = 0
for (u, i) in tqdm(preds.keys()):
    x = preds[u, i] 
    y = user_item2rating[u, i]
    if debug: 
        print('r_hat = %f r = %3f' % (x, y), end=', ')
    acc += (x - y) ** 2 
    count += 1




  0%|          | 0/30654 [00:00<?, ?it/s]


  2%|▏         | 470/30654 [00:00<00:06, 4691.88it/s]




r_hat = 4.085049 r = 4.000000, r_hat = 3.117914 r = 3.000000, r_hat = 2.991131 r = 3.000000, r_hat = 3.149285 r = 3.000000, r_hat = 3.991131 r = 4.000000, r_hat = 4.946396 r = 5.000000, r_hat = 2.150096 r = 2.000000, r_hat = 2.186303 r = 2.000000, r_hat = 2.138829 r = 2.000000, r_hat = 3.094145 r = 3.000000, r_hat = 2.133554 r = 2.000000, r_hat = 3.933888 r = 4.000000, r_hat = 4.933888 r = 5.000000, r_hat = 2.252226 r = 2.000000, r_hat = 5.031052 r = 5.000000, r_hat = 2.259581 r = 2.000000, r_hat = 4.043985 r = 4.000000, r_hat = 3.932086 r = 4.000000, r_hat = 4.071730 r = 4.000000, r_hat = 4.121903 r = 4.000000, r_hat = 3.098664 r = 3.000000, r_hat = 4.046114 r = 4.000000, r_hat = 3.991131 r = 4.000000, r_hat = 2.288201 r = 2.000000, r_hat = 4.991131 r = 5.000000, r_hat = 3.101057 r = 3.000000, r_hat = 3.264787 r = 3.000000, r_hat = 3.264787 r = 3.000000, r_hat = 4.001274 r = 4.000000, r_hat = 3.060306 r = 3.000000, r_hat = 3.991131 r = 4.000000, r_hat = 3.264787 r = 3.000000, r_hat = 

  3%|▎         | 941/30654 [00:00<00:06, 4695.42it/s]


  5%|▍         | 1460/30654 [00:00<00:06, 4831.79it/s]


  6%|▋         | 1961/30654 [00:00<00:05, 4862.26it/s]

r_hat = 5.122276 r = 5.000000, r_hat = 5.147873 r = 5.000000, r_hat = 4.295140 r = 4.000000, r_hat = 5.114330 r = 5.000000, r_hat = 5.202166 r = 5.000000, r_hat = 5.200043 r = 5.000000, r_hat = 4.209304 r = 4.000000, r_hat = 4.325165 r = 4.000000, r_hat = 4.229815 r = 4.000000, r_hat = 4.096295 r = 4.000000, r_hat = 5.123801 r = 5.000000, r_hat = 4.034671 r = 4.000000, r_hat = 2.107516 r = 2.000000, r_hat = 4.094256 r = 4.000000, r_hat = 4.067163 r = 4.000000, r_hat = 4.145230 r = 4.000000, r_hat = 5.202773 r = 5.000000, r_hat = 4.202166 r = 4.000000, r_hat = 5.202166 r = 5.000000, r_hat = 5.141146 r = 5.000000, r_hat = 3.209115 r = 3.000000, r_hat = 4.186670 r = 4.000000, r_hat = 5.120793 r = 5.000000, r_hat = 2.337344 r = 2.000000, r_hat = 1.178996 r = 1.000000, r_hat = 3.294907 r = 3.000000, r_hat = 5.205323 r = 5.000000, r_hat = 3.202166 r = 3.000000, r_hat = 4.099190 r = 4.000000, r_hat = 4.165064 r = 4.000000, r_hat = 3.356390 r = 3.000000, r_hat = 4.221695 r = 4.000000, r_hat = 




  8%|▊         | 2398/30654 [00:00<00:06, 4700.42it/s]


  9%|▉         | 2791/30654 [00:00<00:06, 4369.03it/s]

...

, r_hat = 1.893683 r = 2.000000, r_hat = 2.564247 r = 2.500000, r_hat = 1.365233 r = 1.500000, r_hat = 2.566820 r = 3.000000, r_hat = 0.519029 r = 1.000000, r_hat = 0.397926 r = 1.000000, r_hat = 2.506412 r = 3.000000, r_hat = 2.948068 r = 3.500000, r_hat = 3.219825 r = 3.500000, r_hat = 2.661280 r = 3.000000, r_hat = 1.487787 r = 1.500000, r_hat = 2.770829 r = 3.000000, r_hat = 2.650215 r = 3.000000, r_hat = 3.818145 r = 4.000000, r_hat = 2.637838 r = 3.000000, r_hat = 2.015102 r = 2.500000, r_hat = 3.297795 r = 3.500000, r_hat = 3.999357 r = 4.000000, r_hat = 3.493970 r = 3.500000, r_hat = 3.867044 r = 4.000000, r_hat = 3.125610 r = 3.500000, r_hat = 2.833651 r = 3.000000, r_hat = 1.864427 r = 2.000000, r_hat = 2.549306 r = 3.000000, r_hat = 3.080357 r = 3.500000, r_hat = 3.398626 r = 4.000000, r_hat = 1.418654 r = 1.500000, r_hat = 4.478363 r = 4.500000, r_hat = 2.972355 r = 3.500000, r_hat = 3.003955 r = 3.000000, r_hat = 2.828613 r = 3.000000, r_hat = 2.595159 r = 3.000000, r_hat 




 95%|█████████▍| 29096/30654 [00:06<00:00, 4824.04it/s]


 96%|█████████▋| 29581/30654 [00:06<00:00, 4727.38it/s]

r_hat = 4.358655 r = 4.000000, r_hat = 2.570519 r = 2.500000, r_hat = 4.669573 r = 4.000000, r_hat = 4.109355 r = 4.000000, r_hat = 4.501220 r = 4.500000, r_hat = 3.996680 r = 3.500000, r_hat = 4.355166 r = 4.000000, r_hat = 5.034110 r = 5.000000, r_hat = 3.329547 r = 3.000000, r_hat = 4.496852 r = 4.000000, r_hat = 4.754433 r = 4.500000, r_hat = 3.849463 r = 3.500000, r_hat = 5.087950 r = 5.000000, r_hat = 4.533851 r = 4.500000, r_hat = 5.044795 r = 5.000000, r_hat = 2.740878 r = 2.500000, r_hat = 4.025062 r = 4.000000, r_hat = 4.531532 r = 4.500000, r_hat = 4.299713 r = 4.000000, r_hat = 4.134649 r = 4.000000, r_hat = 5.073461 r = 5.000000, r_hat = 4.497031 r = 4.000000, r_hat = 4.196534 r = 4.000000, r_hat = 4.511564 r = 4.500000, r_hat = 3.244553 r = 3.000000, r_hat = 3.558616 r = 3.000000, r_hat = 3.811796 r = 3.500000, r_hat = 4.860464 r = 5.000000, r_hat = 2.944372 r = 3.000000, r_hat = 3.983299 r = 4.000000, r_hat = 3.862217 r = 3.500000, r_hat = 3.848180 r = 3.500000, r_hat = 




 98%|█████████▊| 30056/30654 [00:06<00:00, 4715.50it/s]


100%|█████████▉| 30529/30654 [00:06<00:00, 4684.56it/s]


100%|██████████| 30654/30654 [00:06<00:00, 4725.43it/s]

, r_hat = 3.280183 r = 4.000000, r_hat = 2.792963 r = 3.000000, r_hat = 3.039400 r = 3.000000, r_hat = 3.091323 r = 3.500000, r_hat = 1.892486 r = 2.000000, r_hat = 1.752412 r = 2.000000, r_hat = 0.504911 r = 0.500000, r_hat = 2.529237 r = 3.000000, r_hat = 2.653028 r = 3.000000, r_hat = 2.991119 r = 3.000000, r_hat = 2.298797 r = 2.500000, r_hat = 2.634808 r = 3.000000, r_hat = 3.242411 r = 3.500000, r_hat = 3.062919 r = 3.500000, r_hat = 3.355205 r = 4.000000, r_hat = 3.413643 r = 4.000000, r_hat = 2.552411 r = 3.000000, r_hat = 3.171160 r = 3.000000, r_hat = 3.066702 r = 3.500000, r_hat = 1.604343 r = 2.000000, r_hat = 3.214594 r = 4.000000, r_hat = 3.253107 r = 3.500000, r_hat = 2.556583 r = 3.000000, r_hat = 1.112613 r = 1.000000, r_hat = 3.570645 r = 4.000000, r_hat = 2.931379 r = 3.500000, r_hat = 2.026009 r = 2.500000, r_hat = 4.144594 r = 4.500000, r_hat = 2.985245 r = 3.000000, r_hat = 1.786668 r = 2.000000, r_hat = 2.522658 r = 3.000000, r_hat = 3.716612 r = 4.000000, r_hat 

In [16]:


print(np.sqrt(acc) / count)

0.001864557100104285
